In [1]:
import os
folder_path=r"C:\Users\Lenovo\Desktop\KG\paper"

for root,dirs,files in os.walk(folder_path):
    for file in files:
        file_path = os.path.abspath(os.path.join(root, file))
        print(file_path)

C:\Users\Lenovo\Desktop\KG\paper\220 kV_3 kA冷绝缘高温超导电缆的轴向温度分布 Longitudinal Temperature Distribution for 220 kV_3 kA CD HTS Cable.pdf
C:\Users\Lenovo\Desktop\KG\paper\A novel method for fast identification of a machine tool selected point temperature rise based on an adaptive unscented Kalman filter.pdf
C:\Users\Lenovo\Desktop\KG\paper\A simulation study on the optimal control of buffeting displacement for the Sutong Bridge with multiple tuned mass dampers.pdf
C:\Users\Lenovo\Desktop\KG\paper\An innovative method to calculate oxygen consumption rate.pdf
C:\Users\Lenovo\Desktop\KG\paper\Analysis on flow characteristics and aerodynamic-force-induced vibration of supersonic rocket-sled.pdf
C:\Users\Lenovo\Desktop\KG\paper\Attitude Control for a Generic Reentry Vehicle via Command Filtered Back-Stepping Technology.pdf
C:\Users\Lenovo\Desktop\KG\paper\Based on RBF neural-network for individual pitch control of wind turbine.pdf
C:\Users\Lenovo\Desktop\KG\paper\Characteristics Analysis of Compr

In [4]:
import os
from openai import OpenAI
from pathlib import Path
import time

client=OpenAI(
    api_key="sk-656b46a1c09c4564b1dc633919201d51",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

folder_path=r"C:\Users\Lenovo\Desktop\KG\paper"

for root,dirs,files in os.walk(folder_path):
    for file in files:
        file_path = os.path.abspath(os.path.join(root, file))
        if not file.lower().endswith(".pdf"):
            continue

        try:
            print(f"正在上传: {file_path} ...")
            file_obj = client.files.create(
                file=Path(file_path),
                purpose="file-extract"  # 表示文件用于内容提取
            )
            print(f"✅ 上传成功: {file_obj.id}")
        except Exception as e:
            print(f"❌ 上传失败 {file_path}: {e}")
        
        # 为防止请求过快，可以加一点延迟（可选）
        time.sleep(1)

正在上传: C:\Users\Lenovo\Desktop\KG\paper\220 kV_3 kA冷绝缘高温超导电缆的轴向温度分布 Longitudinal Temperature Distribution for 220 kV_3 kA CD HTS Cable.pdf ...
✅ 上传成功: file-fe-c3cabcd2c7dd4ebb8c4e4478
正在上传: C:\Users\Lenovo\Desktop\KG\paper\A novel method for fast identification of a machine tool selected point temperature rise based on an adaptive unscented Kalman filter.pdf ...
✅ 上传成功: file-fe-c52749923f03450594b6c0f0
正在上传: C:\Users\Lenovo\Desktop\KG\paper\A simulation study on the optimal control of buffeting displacement for the Sutong Bridge with multiple tuned mass dampers.pdf ...
✅ 上传成功: file-fe-876376eaead549a9b9fb13d9
正在上传: C:\Users\Lenovo\Desktop\KG\paper\An innovative method to calculate oxygen consumption rate.pdf ...
✅ 上传成功: file-fe-fb19496cb4d9455eb9c0fc1f
正在上传: C:\Users\Lenovo\Desktop\KG\paper\Analysis on flow characteristics and aerodynamic-force-induced vibration of supersonic rocket-sled.pdf ...
✅ 上传成功: file-fe-4247d67a285540f6b19e7eb2
正在上传: C:\Users\Lenovo\Desktop\KG\paper\Attitude Con

In [8]:
from openai import OpenAI,BadRequestError
import os
import json

client=OpenAI(
    api_key="sk-656b46a1c09c4564b1dc633919201d51",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

file_list = client.files.list()
file_ids = []

for file in file_list.data:
    file_ids.append(file.id)
    print(file.id)
    
file_refs=",".join([f"fileid://{fid}" for fid in file_ids])

prompt="""请你扮演一位特定领域内专业的知识抽取专家，基于附件内容，从中识别出关键领域实体，并抽取出表达明确逻辑或事实关系的三元组。
三元组格式为：(主体实体, 关系, 客体实体)
具体要求如下：
1. 仅从技术/学术角度出发，抽取相关内容；
2. 实体可以是：模型名称、算法名称、关键技术、数学表达、研究对象、结构参数、性能指标、建模方法、工程应用、实验现象等；
3. 关系词可以是：提出了、用于、导致、依赖于、采用、解决、计算、影响、包括、表示、转换为、对应等动词或动宾短语；
4. 仅输出结构化的 JSON 格式结果,每一行一个 JSON 对象，不输出多余文字、解释、注释或代码块。
"""

try:
    completion=client.chat.completions.create(
        model="deepseek-v3",
        messages=[
            {'role':'system','content':'你是一位特定领域内专业的知识抽取专家'},
            {'role':'system','content':file_refs},
            {'role':'user','content':prompt}
        ],
        stream=True,
        stream_options={"include_usage":True}
    )
    
    full_content=""
    for chunk in completion:
        if chunk.choices and chunk.choices[0].delta.content:
            full_content+=chunk.choices[0].delta.content
            # print(chunk.choices[0].delta.content, end="", flush=True)
    print(full_content)
    print("\n模型输出完成!")
    
    save_path = "extracted_triples.jsonl"
    with open(save_path, "a", encoding="utf-8") as f:
        # 去除 Markdown 代码块符号（模型可能误加 ```json）
        clean_text = full_content.strip().replace("```json", "").replace("```", "").strip()

        # 如果模型输出的是一个 JSON 数组（如 [ {...}, {...} ]），则逐行展开
        try:
            parsed = json.loads(clean_text)
            if isinstance(parsed, list):
                for i, item in enumerate(parsed, 1):
                    if "id" not in item:
                        item["id"] = i
                    f.write(json.dumps(item, ensure_ascii=False) + "\n")
            elif isinstance(parsed, dict):
                f.write(json.dumps(parsed, ensure_ascii=False) + "\n")
            else:
                f.write(clean_text + "\n")
        except json.JSONDecodeError:
            # 若已是 JSONL 格式（每行一个 JSON），直接写入
            f.write(clean_text + "\n")

    print(f"---------------结果已保存到：{save_path}---------------")
except BadRequestError as e:
    print("请求错误，错误信息：",e)


file-fe-5772a67cd4d74559852f1305
file-fe-fe492bef46fe449486623e14
file-fe-d982f9cff33043f5aabe6e63
file-fe-529a388370cf449aa2430269
file-fe-69f0a13e8b1d4eacbb2ac543
file-fe-b3c9f6ad77f949f68dfa03f0
file-fe-58a12d4f36e3485db57632df
file-fe-f5ef5cff4e5b4bb982a1e8af
file-fe-baa4f15cea024d8686d176ea
file-fe-d3a187368c6641ce8d98303c
file-fe-8ba4e7c5362a42d7b7c51283
file-fe-b4cd9127798647e09995d88f
file-fe-198625ec80d0425791117e7a
file-fe-0cb995555b7d438b9beec09e
file-fe-43a32ccf7d7f47a0857f5bfe
file-fe-6bbd59aaccba49659032cf2a
file-fe-b2364059e0cf451c8ae3320d
file-fe-ddb9b25cdde244d88a14ade3
file-fe-fde78e8d7f7b40d6a6e1f84f
file-fe-7a2613e675f9425bb62c685a
file-fe-043b016adf2c42d5990253c2
file-fe-4b2f575d446a4f07b72a03cc
file-fe-9a58ba799c5343e1afd4d85f
file-fe-8d482b990edb49159c97e932
file-fe-a9fbaf377c6448429a83345d
file-fe-7d47863d3f584a9bb2819970
file-fe-ccfe8778d1624592b7ffde92
file-fe-bc8890ee776e4d4da7f17ea9
file-fe-c706e74ed05f4acd8635883c
file-fe-deaa83a6a34b4220a38808b5
file-fe-51

In [1]:
import json
import re

full_content="""
{
  "主体实体": "SMA材料",
  "关系": "具有",
  "客体实体": "形状记忆效应"
}
{
  "主体实体": "SMA材料",
  "关系": "具有",
  "客体实体": "超弹性"
}
{
  "主体实体": "SMA驱动器",
  "关系": "采用",
  "客体实体": "SMA材料"
}
{
  "主体实体": "SMA驱动器",
  "关系": "用于",
  "客体实体": "仿生机器人"
}
{
  "主体实体": "本征热力学模型",
  "关系": "描述",
  "客体实体": "SMA材料"
}
{
  "主体实体": "多尺度方法",
  "关系": "用于",
  "客体实体": "SMA材料建模"
}
{
  "主体实体": "SMA驱动器",
  "关系": "具有",
  "客体实体": "大功率密度"
}
{
  "主体实体": "SMA驱动器",
  "关系": "具有",
  "客体实体": "高能量转换效率"
}
{
  "主体实体": "SMA驱动器",
  "关系": "采用",
  "客体实体": "电热驱动"
}
{
  "主体实体": "SMA驱动器",
  "关系": "存在",
  "客体实体": "响应滞后"
}
{
  "主体实体": "SMA驱动器",
  "关系": "存在",
  "客体实体": "非线性特性"
}
{
  "主体实体": "PID控制器",
  "关系": "用于",
  "客体实体": "SMA驱动器控制"
}
{
  "主体实体": "滑模控制",
  "关系": "用于",
  "客体实体": "SMA驱动器控制"
}
"""
# full_content 是模型的原始输出
pattern = re.compile(r'\{[^{}]*\}', re.S)  # 匹配每个 JSON 对象
matches = pattern.findall(full_content)

triples = []
for idx, m in enumerate(matches, start=1):
    try:
        obj = json.loads(m)
        triple = {
            "head": obj.get("主体实体", "").strip(),
            "relation": obj.get("关系", "").strip(),
            "tail": obj.get("客体实体", "").strip()
        }
        triples.append(triple)
    except json.JSONDecodeError:
        print("跳过无效JSON：", m)

# 保存为 JSONL 文件
with open("triples.jsonl", "w", encoding="utf-8") as f:
    for triple in triples:
        f.write(json.dumps(triple, ensure_ascii=False) + "\n")

print("✅ 已保存为 triples.jsonl，共", len(triples), "条。")


✅ 已保存为 triples.jsonl，共 13 条。


In [3]:
from openai import OpenAI,BadRequestError
import os
import re
import json

client=OpenAI(
    api_key="sk-656b46a1c09c4564b1dc633919201d51",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

file_list = client.files.list()
file_ids = []

for file in file_list.data:
    file_ids.append(file.id)
    print(file.id)
    
file_refs=",".join([f"fileid://{fid}" for fid in file_ids])

prompt="""请你扮演一位特定领域内专业的知识抽取专家，基于附件内容，从中识别出关键领域实体，并抽取出表达明确逻辑或事实关系的三元组。
三元组格式为：(主体实体, 关系, 客体实体)
具体要求如下：
1. 仅从技术/学术角度出发，抽取相关内容；
2. 实体可以是：模型名称、算法名称、关键技术、数学表达、研究对象、结构参数、性能指标、建模方法、工程应用、实验现象等；
3. 关系词可以是：提出了、用于、导致、依赖于、采用、解决、计算、影响、包括、表示、转换为、对应等动词或动宾短语；
4. 仅输出结构化的 JSON 格式结果,每一行一个 JSON 对象，不输出多余文字、解释、注释或代码块。
"""

try:
    completion=client.chat.completions.create(
        model="deepseek-v3",
        messages=[
            {'role':'system','content':'你是一位特定领域内专业的知识抽取专家'},
            {'role':'system','content':file_refs},
            {'role':'user','content':prompt}
        ],
        stream=True,
        stream_options={"include_usage":True}
    )
    
    full_content=""
    for chunk in completion:
        if chunk.choices and chunk.choices[0].delta.content:
            full_content+=chunk.choices[0].delta.content
            # print(chunk.choices[0].delta.content, end="", flush=True)
    print(full_content)
    print("\n模型输出完成!")
    
    pattern = re.compile(r'\{[^{}]*\}', re.S)  # 匹配每个 JSON 对象
    matches = pattern.findall(full_content)

    triples = []
    for idx, m in enumerate(matches, start=1):
        try:
            obj = json.loads(m)
            triple = {
                "head": obj.get("主体实体", "").strip(),
                "relation": obj.get("关系", "").strip(),
                "tail": obj.get("客体实体", "").strip()
            }
            triples.append(triple)
        except json.JSONDecodeError:
            print("跳过无效JSON：", m)

    # 保存为 JSONL 文件
    with open("tri.jsonl", "w", encoding="utf-8") as f:
        for triple in triples:
            f.write(json.dumps(triple, ensure_ascii=False) + "\n")

    print("已保存为 tri.jsonl，共", len(triples), "条。")
except BadRequestError as e:
    print("请求错误，错误信息：",e)


file-fe-5772a67cd4d74559852f1305
file-fe-fe492bef46fe449486623e14
file-fe-d982f9cff33043f5aabe6e63
file-fe-529a388370cf449aa2430269
file-fe-69f0a13e8b1d4eacbb2ac543
file-fe-b3c9f6ad77f949f68dfa03f0
file-fe-58a12d4f36e3485db57632df
file-fe-f5ef5cff4e5b4bb982a1e8af
file-fe-baa4f15cea024d8686d176ea
file-fe-d3a187368c6641ce8d98303c
file-fe-8ba4e7c5362a42d7b7c51283
file-fe-b4cd9127798647e09995d88f
file-fe-198625ec80d0425791117e7a
file-fe-0cb995555b7d438b9beec09e
file-fe-43a32ccf7d7f47a0857f5bfe
file-fe-6bbd59aaccba49659032cf2a
file-fe-b2364059e0cf451c8ae3320d
file-fe-ddb9b25cdde244d88a14ade3
file-fe-fde78e8d7f7b40d6a6e1f84f
file-fe-7a2613e675f9425bb62c685a
file-fe-043b016adf2c42d5990253c2
file-fe-4b2f575d446a4f07b72a03cc
file-fe-9a58ba799c5343e1afd4d85f
file-fe-8d482b990edb49159c97e932
file-fe-a9fbaf377c6448429a83345d
file-fe-7d47863d3f584a9bb2819970
file-fe-ccfe8778d1624592b7ffde92
file-fe-bc8890ee776e4d4da7f17ea9
file-fe-c706e74ed05f4acd8635883c
file-fe-deaa83a6a34b4220a38808b5
file-fe-51